In [1]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import undetected_chromedriver as uc
from selenium_stealth import stealth
from selenium.webdriver.common.by import By

In [116]:
HOME_ADDRESS = "https://historicalnewspapers.lib.purdue.edu/?a=cl&cl=CL1&sp=PE&e=-------en-20--1--txt-txIN--------1"
options = Options()
#options.add_argument("--headless")
driver = uc.Chrome(options=options)
stealth(driver,
                languages=["en-US", "en"],
                vendor="Google Inc.",
                platform="Win32",
                webgl_vendor="Intel Inc.",
                renderer="Intel Iris OpenGL Engine",
                fix_hairline=True
        )

driver.implicitly_wait(5)
driver.get(HOME_ADDRESS)
input("Enter anything when captcha complete")
print(driver.title)

The Purdue Exponent (1889-2017) — Browse by title — Purdue University Newspapers Collection


In [117]:
start_year = 1939

In [93]:
calendar = driver.find_element(By.ID, 'datebrowserrichardtoplevelcalendar')

In [94]:
year = driver.find_element(By.XPATH, f"//h2[contains(text(), '{start_year}')]/following-sibling::div[1]")

In [95]:
first_link = year.find_element(
    By.XPATH,
    ".//ul[contains(@class,'nav') and contains(@class,'nav-pills')]//a[@class='nav-link'][1]"
)

In [98]:
first_link = first_link.get_attribute("href")

In [125]:
driver.get(first_link)
super_links = []

In [126]:
num_years = 6
num_months = 12 * num_years

for month in range(num_months + 1):
    calendar_box = driver.find_element(By.ID, "datebrowserrichardmonthlevelcalendar")
    weeks = calendar_box.find_elements(By.XPATH, "./*[position() > 1]")
    days = calendar_box.find_elements(
        By.XPATH,
        ".//div[@class='datebrowserrichardmonthlevelcalendardaycell' and not(contains(@class, ' '))]"
    )

    for day in days:
        date = day.find_element(By.CSS_SELECTOR, ".hiddenwhennotsmall")
        print(date.get_attribute("innerHTML"))
        link = day.find_element(By.CSS_SELECTOR, "li.list-group-item a")
        href = link.get_attribute("href")
        print(href)
        super_links.append((date.get_attribute("innerHTML"), href))
    next_month = driver.find_element(
        By.XPATH,
        "//a[span[text()='Next month']]"
    )
    href = next_month.get_attribute("href")
    print(href)
    driver.get(href)


Friday, 6 January 1939
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19390106-01&e=-------en-20--1--txt-txIN--------1
Saturday, 7 January 1939
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19390107-01&e=-------en-20--1--txt-txIN--------1
Sunday, 8 January 1939
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19390108-01&e=-------en-20--1--txt-txIN--------1
Tuesday, 10 January 1939
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19390110-01&e=-------en-20--1--txt-txIN--------1
Wednesday, 11 January 1939
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19390111-01&e=-------en-20--1--txt-txIN--------1
Thursday, 12 January 1939
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19390112-01&e=-------en-20--1--txt-txIN--------1
Friday, 13 January 1939
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19390113-01&e=-------en-20--1--txt-txIN--------1
Saturday, 14 January 1939
https://historicalnewspapers.lib.purdue.edu/?a=d&d=PE19390114-01&e=-------en-20--1--txt

In [127]:
import requests
from pathlib import Path

pdf_links = []
for date, super_link in super_links:
    driver.get(super_link)
    pdf_link_element = driver.find_element(By.CLASS_NAME, "pdflink")
    pdf_href = pdf_link_element.get_attribute("href")

    pdf_links.append(pdf_href)
    # container = driver.find_element(By.ID, "documentdisplayleftpanecontentscontainer")
    #
    # links = container.find_elements(By.CSS_SELECTOR, "a.pagetocnodelink")
    # links = [link.get_attribute("href") for link in links]
    #
    # agg_text = ""
    # for link in links:
    #     driver.get(link)
    #     container = driver.find_element(By.ID, "documentdisplayleftpanesectiontextcontainer")
    #
    #     header = container.find_element(
    #         By.CSS_SELECTOR,
    #         "div.documentdisplayleftpanesectiontextheader"
    #     )
    #
    #     paragraphs = container.find_elements(
    #         By.CSS_SELECTOR,
    #         "p[dir='auto']"
    #     )
    #
    #     text = "\n".join([p.text for p in paragraphs])
    #
    #     agg_text += text
    #     driver.get(super_link)
    #
    # with open(f"/home/malinkyzubr/Desktop/HIST302/302hist/files/{date}.txt", "w") as f:
    #     f.write(agg_text)


In [129]:
pdf_links[-1]

'https://historicalnewspapers.lib.purdue.edu/?a=is&oid=PE19450130-01&type=staticpdf&e=-------en-20--1--txt-txIN--------1'

In [105]:
file = "/home/malinkyzubr/Downloads/PE_19390202_01.pdf"

In [107]:
from pdf2image import convert_from_path
import pytesseract

pages = convert_from_path(file)
text = ""
for page in pages:
    text += pytesseract.image_to_string(page)

print(text)

it

"

|

ft
ee

xe

THE WEATHER

Forecast for Indiana: Snow or

rain jn north, rain in south, ris-
ing. temperature.

Purdue. Exp

onent

THURSDAY
February 2, 1939

Lafayette, Indiana

Vol, 50 No. 103

Hold Formal Dance
Tomorrow Evening

The afintal Miami Triad formal
will be held in the Union ballroom
tomrrow evening. Hal Lieber and
his Indiana University orchestra
will furnish the music from 9 to 12
p. m.

The backdrop and ballroom decor-
ations will follow the theme of the
programs which include the crests
of the trio in blue and pink.. The
dance is closed except to members
and their guests.

The Triad is made up of the three
social fraternities-+Phi Delta Theta,
Beta Theta Phi, and Sigma. Chi—
which originated on the Miami Uni-
versity campus in Oxford, O.

While this is not Lieber’s first
appearance on the campus, it is
the first formal dance which he has
played for here. His previous cam-
pus stands include mixers and the
Battle of Swing with Bud Bryant’s
orchestra after the Ind